In [10]:
from pyspark.sql import functions
from pyspark.sql.types import StringType

from c2cbet import config, hlpr
from c2cbet.helper import spark

In [2]:
df = spark.sql("select * from parquet.c2c")
df.show()

+--------------------+--------------------+----------------+-------------------+--------------------+--------------------+--------------------+------+---------------+-------------+---------+--------------+-------------+----------------+-------------------+------------------+---------------------+---------------------+------------+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+----------------+--------------------+--------------+--------------------+
|                 AID|        OWNER_USERID|OWNER_CELLNUMBER|             AMOUNT|        CREATIONDATE|      EXPIRATIONDATE|        EXERCISEDATE|STATUS|      FEECHARGE|         TYPE|INITIATOR|SOURCE_POSTFIX|SOURCE_PREFIX|SOURCE_BANK_CODE|DESTINATION_POSTFIX|DESTINATION_PREFIX|DESTINATION_BANK_CODE|DESTINATION_CARDINDEX|         RRN|              GDATE|                  IP|        TRACKINGCODE|      creationdate_d|creationdate_j|    expirationdate_d|expirationdate_j|      

In [3]:
print(df.count())
df.show()

6059
+--------------------+--------------------+----------------+-------------------+--------------------+--------------------+--------------------+------+---------------+-------------+---------+--------------+-------------+----------------+-------------------+------------------+---------------------+---------------------+------------+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+----------------+--------------------+--------------+--------------------+
|                 AID|        OWNER_USERID|OWNER_CELLNUMBER|             AMOUNT|        CREATIONDATE|      EXPIRATIONDATE|        EXERCISEDATE|STATUS|      FEECHARGE|         TYPE|INITIATOR|SOURCE_POSTFIX|SOURCE_PREFIX|SOURCE_BANK_CODE|DESTINATION_POSTFIX|DESTINATION_PREFIX|DESTINATION_BANK_CODE|DESTINATION_CARDINDEX|         RRN|              GDATE|                  IP|        TRACKINGCODE|      creationdate_d|creationdate_j|    expirationdate_d|expirationdate_j| 

In [5]:
df2 = df.withColumn('count', functions.size(functions.split('ip', ',')))
df2.show(10, False)

+------------------------+------------------------------------+----------------+-------------------+------------------------+------------------------+------------------------+------+---------------+-------------+---------+--------------+-------------+----------------+-------------------+------------------+---------------------+---------------------+------------+-------------------+------------------------------+-----------------------+---------------------+--------------+---------------------+----------------+---------------------+--------------+--------------------+-----+
|AID                     |OWNER_USERID                        |OWNER_CELLNUMBER|AMOUNT             |CREATIONDATE            |EXPIRATIONDATE          |EXERCISEDATE            |STATUS|FEECHARGE      |TYPE         |INITIATOR|SOURCE_POSTFIX|SOURCE_PREFIX|SOURCE_BANK_CODE|DESTINATION_POSTFIX|DESTINATION_PREFIX|DESTINATION_BANK_CODE|DESTINATION_CARDINDEX|RRN         |GDATE              |IP                            |TRACK

In [6]:
df2.groupBy().agg(functions.min('count'), functions.max('count')).show(10, False)


+----------+----------+
|min(count)|max(count)|
+----------+----------+
|2         |2         |
+----------+----------+



In [7]:
df2 = df.withColumn('ip1', functions.split('ip', ',')[0]) \
    .withColumn('ip2', functions.split('ip', ',')[1]) \
    .withColumn('ip3', functions.split('ip', ',')[2])
df2.show(10, False)



+------------------------+------------------------------------+----------------+-------------------+------------------------+------------------------+------------------------+------+---------------+-------------+---------+--------------+-------------+----------------+-------------------+------------------+---------------------+---------------------+------------+-------------------+------------------------------+-----------------------+---------------------+--------------+---------------------+----------------+---------------------+--------------+--------------------+--------------+---------------+----+
|AID                     |OWNER_USERID                        |OWNER_CELLNUMBER|AMOUNT             |CREATIONDATE            |EXPIRATIONDATE          |EXERCISEDATE            |STATUS|FEECHARGE      |TYPE         |INITIATOR|SOURCE_POSTFIX|SOURCE_PREFIX|SOURCE_BANK_CODE|DESTINATION_POSTFIX|DESTINATION_PREFIX|DESTINATION_BANK_CODE|DESTINATION_CARDINDEX|RRN         |GDATE              |IP    

In [17]:
# --Use data from another table
# *****************************
spark.sql("DROP TABLE IF EXISTS c2c_copy")
spark.sql("CREATE TABLE c2c_copy using parquet "
          " AS SELECT * FROM parquet.c2c ")

DataFrame[]

In [18]:
df = spark.sql("select * from c2c_copy")
print(df.count())
df.show()

6059
+--------------------+--------------------+----------------+-------------------+--------------------+--------------------+--------------------+------+---------------+-------------+---------+--------------+-------------+----------------+-------------------+------------------+---------------------+---------------------+------------+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+----------------+--------------------+--------------+--------------------+
|                 AID|        OWNER_USERID|OWNER_CELLNUMBER|             AMOUNT|        CREATIONDATE|      EXPIRATIONDATE|        EXERCISEDATE|STATUS|      FEECHARGE|         TYPE|INITIATOR|SOURCE_POSTFIX|SOURCE_PREFIX|SOURCE_BANK_CODE|DESTINATION_POSTFIX|DESTINATION_PREFIX|DESTINATION_BANK_CODE|DESTINATION_CARDINDEX|         RRN|              GDATE|                  IP|        TRACKINGCODE|      creationdate_d|creationdate_j|    expirationdate_d|expirationdate_j| 

In [12]:
df = spark.sql("select * from parquet.c2c")
# df_bet = spark.sql("select * from parquet.c2c_bet")
df.show()

+--------------------+--------------------+----------------+-------------------+--------------------+--------------------+--------------------+------+---------------+-------------+---------+--------------+-------------+----------------+-------------------+------------------+---------------------+---------------------+------------+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+----------------+--------------------+--------------+--------------------+
|                 AID|        OWNER_USERID|OWNER_CELLNUMBER|             AMOUNT|        CREATIONDATE|      EXPIRATIONDATE|        EXERCISEDATE|STATUS|      FEECHARGE|         TYPE|INITIATOR|SOURCE_POSTFIX|SOURCE_PREFIX|SOURCE_BANK_CODE|DESTINATION_POSTFIX|DESTINATION_PREFIX|DESTINATION_BANK_CODE|DESTINATION_CARDINDEX|         RRN|              GDATE|                  IP|        TRACKINGCODE|      creationdate_d|creationdate_j|    expirationdate_d|expirationdate_j|      

In [34]:
df_bet = spark.sql("select * from parquet.c2c_bet")
# df_bet = spark.sql("select * from parquet.c2c_bet")
df_bet.show()

+----------------+----------------+---------------+------------+----------+------------+------------+---------+----------+----------+-----------+--------------------+-------+----------+-------------+---+------------+------------+
|         pan_src|        pan_dest|transactiontype|terminaltype|      date|        time|         rrn|   amount|merchantno|terminalno|bank_issuer|                 psp|channel|mobile_app|   cellnumber|url|channel_date|channel_time|
+----------------+----------------+---------------+------------+----------+------------+------------+---------+----------+----------+-----------+--------------------+-------+----------+-------------+---+------------+------------+
|6037998141056563|5892101335946055|   کارت به کارت|          59|1400/08/24|15:48:14.346|852959131941|3000000.0| 999999556|  17592030|        ملي|پرداخت الکترونيک ...| فرابوم|      ايوا|9.370229035E9|NaN|         NaN|         NaN|
|6037998170196009|5894631161088893|   کارت به کارت|          59|1400/08/10|10:58

In [43]:
# Expected: string, Found: DOUBLE
# spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
df_bet=df_bet.withColumn("channel",functions.concat_ws(" ",df_bet["channel"],df_bet["channel"]))

In [49]:
# df_bet.groupby("channel").count().show() # error
df_bet=df_bet.drop("channel")


In [52]:
df_bet.groupby("pan_src").count().show() # error
# pan_src|        pan_dest|transactiontype|terminaltype|      date|        time|         rrn|   amount|merchantno|terminalno|bank_issuer|                 psp|channel|mobile_app|   cellnumber|url|channel_date|channel_time

Py4JJavaError: An error occurred while calling o483.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 32.0 failed 1 times, most recent failure: Lost task 7.0 in stage 32.0 (TID 151) (host.docker.internal executor driver): org.apache.spark.sql.execution.QueryExecutionException: Encounter error while reading parquet files. One possible cause: Parquet column cannot be converted in the corresponding files. Details: 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadParquetFilesError(QueryExecutionErrors.scala:577)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:200)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:104)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apa

In [51]:
df_bet.write \
    .mode(config.write_mode) \
    .format(config.write_format)\
    .saveAsTable("c2c_bet")

Py4JJavaError: An error occurred while calling o469.saveAsTable.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.datasources.DataSource.writeAndRead(DataSource.scala:537)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.saveDataIntoTable(createDataSourceTables.scala:228)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:182)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands

In [46]:
spark.sql("DROP TABLE IF EXISTS c2c_bet")

DataFrame[]

In [15]:
spark.sql("CREATE TABLE c2c_bet using parquet "
          " AS SELECT * FROM parquet.c2c_bet")

Py4JJavaError: An error occurred while calling o23.sql.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.datasources.DataSource.writeAndRead(DataSource.scala:537)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.saveDataIntoTable(createDataSourceTables.scala:228)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:182)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:11

In [24]:
hlpr.recreate_spark()

In [22]:
spark.sql("select channel from parquet.c2c_bet").show(300)

Py4JJavaError: An error occurred while calling o87.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 22.0 failed 1 times, most recent failure: Lost task 1.0 in stage 22.0 (TID 38) (host.docker.internal executor driver): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///D:/workspace/digipay-data-science/c2c-bet-fraud/c2cbet/c2c_bet/part-00007-d84ce583-95f1-4ca2-9c10-e3173749b71c-c000.snappy.parquet. Column: [channel], Expected: string, Found: DOUBLE
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:570)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:195)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:104)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:522)
	at org.apache.

In [26]:
spark.sql("create table c2c_bet2 using parquet as select * from parquet.c2c_bet")

Py4JJavaError: An error occurred while calling o23.sql.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.datasources.DataSource.writeAndRead(DataSource.scala:537)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.saveDataIntoTable(createDataSourceTables.scala:228)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:182)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:11